### Acknowledgment
This script was developed with guidance from Ms. Shilpi Shaw, AI Club Research Institute,
and helper code suggestions generated using ChatGPT.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# define the dataset path
TRAIN_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/csv/Select_K_Best_features_train_new.csv"
#TRAIN_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/20feature_csv/Select_K_Best_features_train.csv"
#TRAIN_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/40feature_csv/Select_K_Best_features_train.csv"

In [ ]:
!pip install lifelines torchtuples

In [ ]:
import pandas as pd
import numpy as np
# read the data
data = pd.read_csv(TRAIN_PATH)
data.head()

,ABHD1,AMY2A,ASMTL,FRA10AC1,CCDC7,PEAK3,CD81,CSGALNACT2,DNAJC30,FAM35A,...,SNORA36B,ZBTB6,ZNF22,ZNF248,FLJ23404,ZNF510,ZNF676,ZNF98,Disease Free (Months),DFS_STATUS_ENCODED
0,0.5427,0.0042,0.0727,-0.2001,-2.4443,0.5504,1.3306,-1.0626,0.6281,0.0990,...,-47.5219,-0.3621,-0.7499,-1.2925,-0.3176,-0.0527,-1.1132,-0.3769,8.41,1.0
1,1.0479,-1.6920,-1.1518,0.5470,-1.0024,0.4532,-0.3876,0.5574,0.7236,-0.2224,...,-47.5219,-0.8513,-1.4436,0.4544,0.5857,-0.7763,-0.5119,-0.1707,2.56,1.0
2,-0.3481,-0.0009,-1.4332,-0.7239,-0.2558,-0.1502,-0.2253,0.8611,-0.9020,0.5945,...,-47.5219,-0.3803,0.0617,-0.0401,0.2981,0.5104,-0.5328,-0.5964,1.22,0.0
3,0.8966,-0.7667,0.7776,0.0019,0.3988,0.9589,-0.1055,-0.7851,-0.1803,-0.6278,...,-47.5219,-0.1266,-0.8905,-0.2346,0.3443,0.2540,-0.7956,-1.2834,13.40,1.0
4,0.5154,-1.6920,0.4461,1.4794,-0.4612,-0.4449,-1.3079,1.5391,0.4500,0.0604,...,-47.5219,-0.5663,0.2724,1.1534,1.3512,0.4122,-0.8140,-0.9292,3.19,1.0


In [ ]:
#Drop cols based on convergence

columns_to_drop = ['KRTAP9-4', 'OR2T2', 'OR4F17', 'SNORA36B']
data.drop(columns_to_drop, axis=1, inplace = True)

Creating deep survival model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtuples import Model
import torchtuples as tt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

In [ ]:
import torch
import torch.nn as nn

# Define the DeepSurv model with additional layers and dropout
class DeepSurv(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_prob=0.5):
        super(DeepSurv, self).__init__()
        self.layers = nn.ModuleList()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Create hidden layers with dropout
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            self.layers.append(nn.Linear(prev_dim, hidden_dim))
            prev_dim = hidden_dim

        # Output layer
        self.output_layer = nn.Linear(hidden_dims[-1], 1)

    def forward(self, x):
        # Pass through each hidden layer with ReLU activation and dropout
        for layer in self.layers:
            x = self.relu(layer(x))
            x = self.dropout(x)  # Apply dropout after activation
        return self.output_layer(x)

# Function to get the loss (Negative Log-Likelihood)
def nll_loss(preds, durations, events):
    # Cox proportional hazards loss function
    hazards = preds.squeeze()
    log_hazards = torch.log(torch.cumsum(torch.exp(hazards), dim=0))
    uncensored_loss = hazards - log_hazards
    censored_loss = uncensored_loss * events
    return -torch.sum(censored_loss)


Separate x and y

In [ ]:
DEPENDENT_COLUMNS = ['Disease Free (Months)', 'DFS_STATUS_ENCODED']
GENES = [cols for cols in list(data.columns) if cols not in DEPENDENT_COLUMNS]
#prepare the dataset
X = data[GENES]
y = data[DEPENDENT_COLUMNS]

In [ ]:
X.shape

(73, 36)

Cross validation

In [ ]:
# import numpy as np
# from sklearn.model_selection import KFold
# from tqdm.notebook import tqdm

# #change epoch and learning rate and record mean c-index value in hyperparameter sheet

# HIDDEN_DIMENSION = 64
# #HIDDEN_DIMENSION = [128, 64, 32]
# NO_EPOCHS = 50
# LEARNING_RATE = 0.001

# conco_index = []

# # KFold Cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# for train_index, valid_index in kf.split(X):
#     X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
#     y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

#     # Convert data to tensors
#     X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
#     y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
#     y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#     X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
#     y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
#     y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#     # define the model and the optimizer
#     input_dim = X.shape[1]
#     model = DeepSurv(input_dim, HIDDEN_DIMENSION)
#     optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

#     # train the model
#     for epoch in tqdm(range(NO_EPOCHS)):
#         model.train()
#         optimizer.zero_grad()

#         # Forward pass
#         preds = model(X_train_tensor)

#         # Compute the loss
#         loss = nll_loss(preds, y_train_duration, y_train_event)
#         loss.backward()
#         optimizer.step()

#     # Evaluate the model
#     model.eval()
#     with torch.no_grad():
#         test_preds = model(X_test_tensor)
#         c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
#         conco_index.append(c_index)
#         print(f'Concordance Index: {c_index:.4f}')

# # print evaluations
# print("Cross Validation C-Indexes: {}".format(conco_index))
# print("Mean C-Index: {}".format(np.array(conco_index).mean()))

In [ ]:
import os
# Save the entire model to a file
#SAVE_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurvModel"
#SAVE_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/20feature_DeepSurvModel"
SAVE_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/40feature_DeepSurvModel"
MODEL_NAME = "selectkbest_DeepSurv_try2.pth"
# save the entire model with architecture
#torch.save(model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

# Define parameters for experimentation
epochs_list = [50, 60, 70, 80, 90, 100]
learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
#hidden_layers = [256, 128, 64, 32]
hidden_layers = [128, 64, 32]
#hidden_layers = [128, 64]
#hidden_layers = [64, 32]


# List to store results
results = []

# Variable to track the best model
best_model = None
best_model_state = None
best_c_index = -np.inf  # Initialize with negative infinity
best_params = None  # To store parameters of the best model

# Experiment loop
for epochs in epochs_list:
    for lr in learning_rates:
        conco_index = []  # To store c-index for cross-validation

        # KFold Cross-validation
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        # KFold Cross-validation
        for train_index, valid_index in kf.split(X):
            X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
            y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

            # Convert data to tensors
            X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
            y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
            y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

            X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
            y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
            y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

            # Define the model and optimizer
            input_dim = X.shape[1]
            model = DeepSurv(input_dim, hidden_layers)
            optimizer = optim.Adam(model.parameters(), lr=lr)

            # Train the model
            for epoch in range(epochs):
                model.train()
                optimizer.zero_grad()

                # Forward pass
                preds = model(X_train_tensor)

                # Compute the loss
                loss = nll_loss(preds, y_train_duration, y_train_event)
                loss.backward()
                optimizer.step()

            # Evaluate the model
            model.eval()
            with torch.no_grad():
                test_preds = model(X_test_tensor)
                c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
                conco_index.append(c_index)

        # Record the mean concordance index for this configuration
        mean_c_index = np.mean(conco_index)
        results.append({
            'Epochs': epochs,
            'Learning Rate': lr,
            'Mean C-Index': mean_c_index
        })
        print(f'Epochs: {epochs}, Learning Rate: {lr}, Mean C-Index: {mean_c_index:.4f}')

        # Update the best model if this configuration is the best so far
        if mean_c_index > best_c_index:
            best_c_index = mean_c_index
            best_model_state = model.state_dict()  # Save the model's state dictionary
            best_model = model
            best_params = {'Epochs': epochs, 'Learning Rate': lr}

# Save the best model to a .pth file
torch.save(best_model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))
print(f"Best model saved with C-Index: {best_c_index:.4f} and parameters: {best_params}")

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurv/deepsurv_SKB_results_try1.csv', index=False)
print("Experimentation complete. Results saved to 'deepsurv_SKB_results_try1.csv'.")
#results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/20feature_DeepSurv/deepsurv_SKB_results_try8.csv', index=False)
#print("Experimentation complete. Results saved to 'deepsurv_SKB_results_try8.csv'.")
#results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/40feature_DeepSurv/deepsurv_SKB_results_try2.csv', index=False)
#print("Experimentation complete. Results saved to 'deepsurv_SKB_results_try2.csv'.")



Epochs: 50, Learning Rate: 1e-05, Mean C-Index: 0.5348
Epochs: 50, Learning Rate: 5e-05, Mean C-Index: 0.5405
Epochs: 50, Learning Rate: 0.0001, Mean C-Index: 0.6192
Epochs: 50, Learning Rate: 0.0005, Mean C-Index: 0.6862
Epochs: 50, Learning Rate: 0.001, Mean C-Index: 0.6464
Epochs: 60, Learning Rate: 1e-05, Mean C-Index: 0.5628
Epochs: 60, Learning Rate: 5e-05, Mean C-Index: 0.5242
Epochs: 60, Learning Rate: 0.0001, Mean C-Index: 0.5863
Epochs: 60, Learning Rate: 0.0005, Mean C-Index: 0.6705
Epochs: 60, Learning Rate: 0.001, Mean C-Index: 0.6255
Epochs: 70, Learning Rate: 1e-05, Mean C-Index: 0.5078
Epochs: 70, Learning Rate: 5e-05, Mean C-Index: 0.5267
Epochs: 70, Learning Rate: 0.0001, Mean C-Index: 0.6724
Epochs: 70, Learning Rate: 0.0005, Mean C-Index: 0.6509
Epochs: 70, Learning Rate: 0.001, Mean C-Index: 0.6332
Epochs: 80, Learning Rate: 1e-05, Mean C-Index: 0.5431
Epochs: 80, Learning Rate: 5e-05, Mean C-Index: 0.6129
Epochs: 80, Learning Rate: 0.0001, Mean C-Index: 0.6496
Epo

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.model_selection import KFold
# from lifelines.utils import concordance_index

# # Define parameters for experimentation
# epochs_list = [50, 60, 70, 80, 90, 100]
# learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
# hidden_layers_list = [
#     [128, 64, 32],
#     [64, 32],
#     [64]
# ]
# dropout_rates = [0, 0.2, 0.3, 0.4, 0.5]

# # List to store results
# results = []

# # Variable to track the best model
# best_model = None
# best_model_state = None
# best_c_index = -np.inf  # Initialize with negative infinity
# best_params = None  # To store parameters of the best model

# # Experiment loop
# for hidden_layers in hidden_layers_list:
#     for dropout_rate in dropout_rates:
#         for epochs in epochs_list:
#             for lr in learning_rates:
#                 conco_index = []  # To store c-index for cross-validation

#                 # KFold Cross-validation
#                 kf = KFold(n_splits=5, shuffle=True, random_state=42)

#                 for train_index, valid_index in kf.split(X):
#                     X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
#                     y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

#                     # Convert data to tensors
#                     X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
#                     y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
#                     y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#                     X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
#                     y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
#                     y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#                     # Define the model and optimizer
#                     input_dim = X.shape[1]
#                     model = DeepSurv(input_dim, hidden_layers, dropout_rate)
#                     optimizer = optim.Adam(model.parameters(), lr=lr)

#                     # Train the model
#                     for epoch in range(epochs):
#                         model.train()
#                         optimizer.zero_grad()

#                         # Forward pass
#                         preds = model(X_train_tensor)

#                         # Compute the loss
#                         loss = nll_loss(preds, y_train_duration, y_train_event)
#                         loss.backward()
#                         optimizer.step()

#                     # Evaluate the model
#                     model.eval()
#                     with torch.no_grad():
#                         test_preds = model(X_test_tensor)
#                         c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
#                         conco_index.append(c_index)

#                 # Record the mean concordance index for this configuration
#                 if conco_index:  # Ensure the list is not empty
#                     mean_c_index = np.mean(conco_index)
#                     results.append({
#                         'Hidden Layers': hidden_layers,
#                         'Dropout Rate': dropout_rate,
#                         'Epochs': epochs,
#                         'Learning Rate': lr,
#                         'Mean C-Index': mean_c_index
#                     })
#                     print(f'Hidden Layers: {hidden_layers}, Dropout Rate: {dropout_rate}, Epochs: {epochs}, '
#                           f'Learning Rate: {lr}, Mean C-Index: {mean_c_index:.4f}')

#                     # Update the best model if this configuration is the best so far
#                     if mean_c_index > best_c_index:
#                         best_c_index = mean_c_index
#                         best_model_state = model.state_dict()  # Save the model's state dictionary
#                         best_model = model
#                         best_params = {
#                             'Hidden Layers': hidden_layers,
#                             'Dropout Rate': dropout_rate,
#                             'Epochs': epochs,
#                             'Learning Rate': lr
#                         }

# # Save the best model to a .pth file
# torch.save(best_model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))
# print(f"Best model saved with C-Index: {best_c_index:.4f} and parameters: {best_params}")

# # Convert results to DataFrame and save to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/20feature_DeepSurv/deepsurv_SKB_results.csv', index=False)
# print("Experimentation complete. Results saved to 'deepsurv_SKB_results.csv'.")
